# fastMRI
Python 3.6+, Tensorflow 2.0.0-dev20190601

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, glob
import h5py # for loading .h5 files
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
plt.style.use('seaborn') # pretty matplotlib plots
plt.rcParams["axes.grid"] = False # don't show grid lines on plots by default
plt.rcParams['figure.figsize'] = (12, 16) # increase size of subplots

In [4]:
import cs230_project_utilities as utils

In [5]:
# Sync data:
# aws s3 sync s3://cs230-automap/data/fastmri/raw/singlecoil_val data/fastmri/raw/singlecoil_dev # (val -> dev) 
# aws s3 sync s3://cs230-automap/data/fastmri/raw/singlecoil_test data/fastmri/raw/singlecoil_test
# aws s3 sync s3://cs230-automap/data/fastmri/raw/singlecoil_train data/fastmri/raw/singlecoil_train

In [6]:
# Prepare local directories

raw_dev_locations = '/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/*.h5'
raw_test_locations = '/home/ubuntu/cs230/data/fastmri/raw/singlecoil_test/*.h5'
raw_train_locations = '/home/ubuntu/cs230/data/fastmri/raw/singlecoil_train/*.h5'

dev_tfrecord_directory = '/home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_dev'
test_tfrecord_directory = '/home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_test'
train_tfrecord_directory = '/home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_train'

!mkdir -p $dev_tfrecord_directory
!mkdir -p $test_tfrecord_directory
!mkdir -p $train_tfrecord_directory

In [7]:
# Run conversion

utils.fastmri.convert_fastmri_dataset_to_tfrecord_files(raw_dev_locations,
                                                        dev_tfrecord_directory,
                                                        coils='single',
                                                        keep_original_reconstruction=True,
                                                        perform_subsampling=False,
                                                        perform_normalization=True)

Converting file 0 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1000480.h5' to TFRecords...
(-63.14653+62.16977j)
(217.87761+10.2137985j)
(459.65585-46.80155j)
(595.6106+1.9383194j)
(963.24414-5.665181j)
(1264.7317-16.838491j)
(1420.0876+24.616272j)
(1245.9425+81.51757j)
(1592.6876+153.65582j)
(1968.8383+135.7207j)
(2921.9744+205.72762j)
(2560.7778+125.6308j)
(2296.1042+70.94549j)
(3084.3604+122.96695j)
(2535.907+114.88403j)
(2429.8481+46.861343j)
(2756.4983+115.0078j)
(2918.4658+79.415665j)
(2510.5247+0.059951782j)
(2865.9614-44.127083j)
(2752.3342+30.712692j)
(2846.4355+52.4375j)
(3224.933-4.411598j)
(2958.021-97.042755j)
(3369.8643-152.3348j)
(3396.815-160.66156j)
(4733.125-189.22583j)
(4936.3125-241.91867j)
(5652.161-271.67487j)
(4761.418-271.93878j)
(4677.969-367.2965j)
(4567.0063-327.8805j)
(4274.333-329.5691j)
Converting file 1 at b'/home/ubuntu/cs230/data/fastmri/raw/singlecoil_dev/file1001170.h5' to TFRecords...
(44.36642+219.1896j)
(-53.862648+404.71863j)
(-39.9

KeyboardInterrupt: 

In [ ]:
utils.fastmri.convert_fastmri_dataset_to_tfrecord_files(raw_test_locations,
                                                        test_tfrecord_directory,
                                                        coils='single',
                                                        keep_original_reconstruction=False,
                                                        perform_subsampling=False,
                                                        perform_normalization=True)

In [ ]:
utils.fastmri.convert_fastmri_dataset_to_tfrecord_files(raw_train_locations,
                                                        train_tfrecord_directory,
                                                        coils='single',
                                                        keep_original_reconstruction=False,
                                                        perform_subsampling=False,
                                                        perform_normalization=True)

In [ ]:
# Sync TFRecords to s3
# aws s3 sync /home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_dev s3://cs230-automap/data/fastmri/tfrecords/singlecoil_dev
# aws s3 sync /home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_test s3://cs230-automap/data/fastmri/tfrecords/singlecoil_test
# aws s3 sync /home/ubuntu/cs230/data/fastmri/tfrecords/singlecoil_train s3://cs230-automap/data/fastmri/tfrecords/singlecoil_train

# Inspect dev data

In [ ]:
# Load data

data_locations = os.path.join(dev_tfrecord_directory, 'shard-0.tfrecord')
dataset = utils.fastmri.load_dataset(data_locations,
                                     batch_size=8,
                                     shuffle_buffer_size=40,
                                     load_original_reconstruction=False,
                                     include_all_parsed_features=False,
                                     ignore_errors=True,
                                     perform_data_augmentation=False)

# Get first batch
batch = next(iter(dataset))
x, y = batch
x, y = x[0].numpy().squeeze(), y[0].numpy().squeeze()


# For visualization, combine 2 channels of complex tensor into single channel
x = utils.fastmri._combine_two_channels_of_complex_tensor(x).numpy()

x.shape, y.shape

In [ ]:
plt.title('FFT')
utils.plot.imshowfft(x)

In [ ]:
plt.title('Reconstruction')
utils.plot.imshowgray(y)

# Inspect test data

In [ ]:
# Load data

data_locations = os.path.join(test_tfrecord_directory, 'shard-0.tfrecord')
dataset = utils.fastmri.load_dataset(data_locations,
                                     batch_size=8,
                                     shuffle_buffer_size=40,
                                     load_original_reconstruction=False,
                                     include_all_parsed_features=False,
                                     ignore_errors=True,
                                     perform_data_augmentation=False)

# Get first batch
batch = next(iter(dataset))
x, y = batch
x, y = x[0].numpy().squeeze(), y[0].numpy().squeeze()


# For visualization, combine 2 channels of complex tensor into single channel
x = utils.fastmri._combine_two_channels_of_complex_tensor(x).numpy()

x.shape, y.shape

In [ ]:
plt.title('FFT')
utils.plot.imshowfft(x)

In [ ]:
plt.title('Reconstruction')
utils.plot.imshowgray(y)

# Inspect train data

In [ ]:
# Load data

data_locations = os.path.join(train_tfrecord_directory, 'shard-0.tfrecord')
dataset = utils.fastmri.load_dataset(data_locations,
                                     batch_size=8,
                                     shuffle_buffer_size=40,
                                     load_original_reconstruction=False,
                                     include_all_parsed_features=False,
                                     ignore_errors=True,
                                     perform_data_augmentation=False)

# Get first batch
batch = next(iter(dataset))
x, y = batch
x, y = x[0].numpy().squeeze(), y[0].numpy().squeeze()


# For visualization, combine 2 channels of complex tensor into single channel
x = utils.fastmri._combine_two_channels_of_complex_tensor(x).numpy()

x.shape, y.shape

In [ ]:
plt.title('FFT')
utils.plot.imshowfft(x)

In [ ]:
plt.title('Reconstruction')
utils.plot.imshowgray(y)